In [ ]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
classes = []
with open('cfg/coco.names', 'r') as f:
    classes = f.read().splitlines()

# Load input video
video = cv2.VideoCapture('videos/sst_foyer_1.mp4')

# Get video properties
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, height))

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Preprocess input frame
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (224, 224), swapRB=True, crop=False)
    #blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Forward pass through the network
    output_layers_names = net.getUnconnectedOutLayersNames()
    layer_outputs = net.forward(output_layers_names)

    # Process detection results
    boxes = []
    confidences = []
    class_ids = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-max suppression to remove redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    # Draw bounding boxes and labels
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

    # Write the frame with detected objects to the output video
    out.write(frame)

    # Display the frame
    cv2.imshow('Video', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and writer objects
video.release()
out.release()
cv2.destroyAllWindows()


In [20]:
## CODEE !!
import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog

# Function to play video normally
def play_video():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Video', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to perform object detection on video
def detect_objects():
    net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
    classes = []
    with open('cfg/coco.names', 'r') as f:
        classes = f.read().splitlines()

    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (224, 224), swapRB=True, crop=False)
        #blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        output_layers_names = net.getUnconnectedOutLayersNames()
        layer_outputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        if len(indexes) > 0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i], 2))
                color = colors[i]
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

        out.write(frame)
        cv2.imshow('Video with Object Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Function to perform operations based on user choice
def operation_window():
    name = entry_name.get()
    messagebox.showinfo("Welcome", f"Welcome {name}. \nThis application allows you to perform object detection on a video.")
    home_window.destroy()
    
    operation_window = tk.Tk()   
    operation_window.title("Choose Operation")

    #label
    operation_label = tk.Label(operation_window, text="Select Operation an operation you would like to perform:")
    operation_label.pack()

    #radio button
    operation_var = tk.IntVar()
    tk.Radiobutton(operation_window, text="Watch Video", variable=operation_var, value=1).pack()
    tk.Radiobutton(operation_window, text="Perform Object Detection", variable=operation_var, value=2).pack()
    video_label = tk.Label(operation_window, text="You can press Q to stop the video at any time")

    #Button
    operation_button = tk.Button(operation_window, text="Proceed", command=lambda: perform_operation(operation_var.get(), operation_window))
    operation_button.pack()

def perform_operation(operation, operation_window):
    operation_window.destroy()
    if operation == 1:
        play_video()
    else:
       detect_objects()

        
# Function to reset the application
def reset():
    entry_name.delete(0, tk.END)
    messagebox.showinfo("Reset", "Application has been reset.")

# Create tkinter home_window
home_window = tk.Tk()
home_window.title("Video Processing Application")

# Create widgets
label_name = tk.Label(home_window, text="Enter your name:")
entry_name = tk.Entry(home_window)
button_start = tk.Button(home_window, text="Start", command=operation_window)
button_reset = tk.Button(home_window, text="Reset", command=reset)

# Place widgets in the home_window
label_name.grid(row=0, column=0, padx=10, pady=5)
entry_name.grid(row=0, column=1, padx=10, pady=5)
button_start.grid(row=1, column=0, columnspan=2, padx=10, pady=5, sticky="WE")
button_reset.grid(row=2, column=0, columnspan=2, padx=10, pady=5, sticky="WE")

# Run the tkinter event loop
home_window.mainloop()

In [10]:
## First draft with multiple operations

import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog

# Function to play video normally
def play_video():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Video', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to perform object detection on video
def detect_objects():
    net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
    classes = []
    with open('cfg/coco.names', 'r') as f:
        classes = f.read().splitlines()

    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        output_layers_names = net.getUnconnectedOutLayersNames()
        layer_outputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        if len(indexes) > 0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i], 2))
                color = colors[i]
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

        out.write(frame)
        cv2.imshow('Video with Object Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Function to reduce noise in the video
def reduce_noise():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    out_path = 'output_videos/reduced_noise_video.avi'
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(out_path, fourcc, 20.0, (640, 480))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        filtered_frame = cv2.medianBlur(frame, 15)  # Apply median blur
        out.write(filtered_frame)
        cv2.imshow('Reduced Noise Video', filtered_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Function to sharpen the video
def sharpen_video():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    out_path = 'output_videos/sharpened_video.avi'
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(out_path, fourcc, 20.0, (640, 480))

    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])  # Sharpening kernel

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        sharpened_frame = cv2.filter2D(frame, -1, kernel)  # Apply sharpening
        out.write(sharpened_frame)
        cv2.imshow('Sharpened Video', sharpened_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Function to adjust brightness and contrast in the video
def adjust_brightness_contrast():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    out_path = 'output_videos/adjusted_video.avi'
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(out_path, fourcc, 20.0, (640, 480))

    brightness = 5
    contrast = 1.5

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        adjusted_frame = cv2.addWeighted(frame, contrast, np.zeros(frame.shape, frame.dtype), 0, brightness)
        out.write(adjusted_frame)
        cv2.imshow('Adjusted Video', adjusted_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Function to perform operations based on user choice
def operation_window():
    name = entry_name.get()
    messagebox.showinfo("Welcome", f"Welcome {name}. \nThis application allows you to perform various operations on a video.")
    home_window.destroy()
    
    operation_window = tk.Tk()   
    operation_window.title("Choose Operation")

    # Label
    operation_label = tk.Label(operation_window, text="Select an operation you would like to perform:")
    operation_label.pack()

    # Radio buttons for operations
    tk.Radiobutton(operation_window, text="Watch Video", variable=operation_var, value=1).pack()
    tk.Radiobutton(operation_window, text="Perform Object Detection", variable=operation_var, value=2).pack()
    tk.Radiobutton(operation_window, text="Reduce Noise", variable=operation_var, value=3).pack()
    tk.Radiobutton(operation_window, text="Sharpen Video", variable=operation_var, value=4).pack()
    tk.Radiobutton(operation_window, text="Adjust Brightness and Contrast", variable=operation_var, value=5).pack()
    
    # Button
    operation_button = tk.Button(operation_window, text="Proceed", command=lambda: perform_operation(operation_var.get(), operation_window))
    operation_button.pack()

def perform_operation(operation, operation_window):
    operation_window.destroy()
    if operation == 1:
        play_video()
    elif operation == 2:
       detect_objects()
    elif operation == 3:
        reduce_noise()
    elif operation == 4:
        sharpen_video()
    elif operation == 5:
        adjust_brightness_contrast()

# Function to reset the application
def reset():
    entry_name.delete(0, tk.END)
    messagebox.showinfo("Reset", "Application has been reset.")

# Create tkinter home_window
home_window = tk.Tk()
home_window.title("Video Processing Application")

# Create widgets
label_name = tk.Label(home_window, text="Enter your name:")
entry_name = tk.Entry(home_window)
button_start = tk.Button(home_window, text="Start", command=operation_window)
button_reset = tk.Button(home_window, text="Reset", command=reset)

# Place widgets in the home_window
label_name.grid(row=0, column=0, padx=10, pady=5)
entry_name.grid(row=0, column=1, padx=10, pady=5)
button_start.grid(row=1, column=0, columnspan=2, padx=10, pady=5, sticky="WE")
button_reset.grid(row=2, column=0, columnspan=2, padx=10, pady=5, sticky="WE")

# Variable to store operation choice
operation_var = tk.IntVar()

# Run the tkinter event loop
home_window.mainloop()


In [14]:
##Good draft with all the operations
import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog

# Function to play video normally
def play_video():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Video', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to perform object detection on video
def detect_objects():
    net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
    classes = []
    with open('cfg/coco.names', 'r') as f:
        classes = f.read().splitlines()

    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        output_layers_names = net.getUnconnectedOutLayersNames()
        layer_outputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    w = int(detection[2] * frame.shape[1])
                    h = int(detection[3] * frame.shape[0])

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        if len(indexes) > 0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i], 2))
                color = colors[i]
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

        cv2.imshow('Video with Object Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to reduce noise on video
def reduce_noise():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        filtered_frame = cv2.medianBlur(frame, 15)  # Apply median blur
        cv2.imshow('Reduced Noise Video', filtered_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to sharpen video
def sharpen_video():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])  # Sharpening kernel

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        sharpened_frame = cv2.filter2D(frame, -1, kernel)  # Apply sharpening
        cv2.imshow('Sharpened Video', sharpened_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to adjust brightness and contrast on video
def adjust_brightness_contrast():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    brightness = 5
    contrast = 1.5

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        adjusted_frame = cv2.addWeighted(frame, contrast, np.zeros(frame.shape, frame.dtype), 0, brightness)
        cv2.imshow('Adjusted Video', adjusted_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to perform operations based on user choice
def operation_window():
    name = entry_name.get()
    messagebox.showinfo("Welcome", f"Welcome {name}. \nThis application allows you to perform various operations on a video.")
    home_window.destroy()
    
    operation_window = tk.Tk()   
    operation_window.title("Choose Operation")

    #label
    operation_label = tk.Label(operation_window, text="Select an operation you would like to perform:")
    operation_label.pack()

    #radio button
    operation_var = tk.IntVar()
    tk.Radiobutton(operation_window, text="Watch Video Normally", variable=operation_var, value=1).pack()
    tk.Radiobutton(operation_window, text="Perform Object Detection", variable=operation_var, value=2).pack()
    tk.Radiobutton(operation_window, text="Reduce Noise", variable=operation_var, value=3).pack()
    tk.Radiobutton(operation_window, text="Sharpen Video", variable=operation_var, value=4).pack()
    tk.Radiobutton(operation_window, text="Adjust Brightness and Contrast", variable=operation_var, value=5).pack()

    #Button
    operation_button = tk.Button(operation_window, text="Proceed", command=lambda: perform_operation(operation_var.get(), operation_window))
    operation_button.pack()

def perform_operation(operation, operation_window):
    operation_window.destroy()
    if operation == 1:
        play_video()
    elif operation == 2:
        detect_objects()
    elif operation == 3:
        reduce_noise()
    elif operation == 4:
        sharpen_video()
    elif operation == 5:
        adjust_brightness_contrast()

# Function to reset the application
def reset():
    entry_name.delete(0, tk.END)
    messagebox.showinfo("Reset", "Application has been reset.")

# Create tkinter home_window
home_window = tk.Tk()
home_window.title("Video Processing Application")

# Create widgets
label_name = tk.Label(home_window, text="Enter your name:")
entry_name = tk.Entry(home_window)
button_start = tk.Button(home_window, text="Start", command=operation_window)
button_reset = tk.Button(home_window, text="Reset", command=reset)

# Place widgets in the home_window
label_name.grid(row=0, column=0, padx=10, pady=5)
entry_name.grid(row=0, column=1, padx=10, pady=5)
button_start.grid(row=1, column=0, columnspan=2, padx=10, pady=5, sticky="WE")
button_reset.grid(row=2, column=0, columnspan=2, padx=10, pady=5, sticky="WE")

# Run the tkinter event loop
home_window.mainloop()


In [19]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox

# Function to play video normally
def play_video():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Video', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to perform object detection on video
def detect_objects():
    net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
    classes = []
    with open('cfg/coco.names', 'r') as f:
        classes = f.read().splitlines()

    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (224, 224), swapRB=True, crop=False)
        net.setInput(blob)
        output_layers_names = net.getUnconnectedOutLayersNames()
        layer_outputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        if len(indexes) > 0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i], 2))
                color = colors[i]
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

        out.write(frame)
        cv2.imshow('Video with Object Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Function to reduce noise in the video
def reduce_noise():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        filtered_image = cv2.medianBlur(frame, 15)
        cv2.imshow('Video with Reduced Noise', filtered_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to sharpen the video
def sharpen_video():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened_image = cv2.filter2D(frame, -1, kernel)
        cv2.imshow('Sharpened Video', sharpened_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to adjust brightness and contrast of the video
def adjust_brightness_contrast():
    cap = cv2.VideoCapture('videos/sst_foyer_1.mp4')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        brightness = 5
        contrast = 1.5
        adjusted_image = cv2.addWeighted(frame, contrast, np.zeros(frame.shape, frame.dtype), 0, brightness)
        cv2.imshow('Video with Adjusted Brightness and Contrast', adjusted_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to perform operations based on user choice
def operation_window():
    name = entry_name.get()
    messagebox.showinfo("Welcome", f"Welcome {name}. This application allows you to perform various operations on a video.")
    home_window.destroy()
    
    operation_window = tk.Tk()   
    operation_window.title("Choose Operation")

    #label
    operation_label = tk.Label(operation_window, text="Select an operation you would like to perform:")
    operation_label.pack()

    #radio button
    operation_var = tk.IntVar()
    tk.Radiobutton(operation_window, text="Watch Video Normally", variable=operation_var, value=1).pack()
    tk.Radiobutton(operation_window, text="Perform Object Detection", variable=operation_var, value=2).pack()
    tk.Radiobutton(operation_window, text="Reduce Noise", variable=operation_var, value=3).pack()
    tk.Radiobutton(operation_window, text="Sharpen Video", variable=operation_var, value=4).pack()
    tk.Radiobutton(operation_window, text="Adjust Brightness and Contrast", variable=operation_var, value=5).pack()

    #Button
    operation_button = tk.Button(operation_window, text="Proceed", command=lambda: perform_operation(operation_var.get(), operation_window))
    operation_button.pack()

def perform_operation(operation, operation_window):
    operation_window.destroy()
    if operation == 1:
        play_video()
    elif operation == 2:
        detect_objects()
    elif operation == 3:
        reduce_noise()
    elif operation == 4:
        sharpen_video()
    elif operation == 5:
        adjust_brightness_contrast()

# Function to reset the application
def reset():
    entry_name.delete(0, tk.END)
    messagebox.showinfo("Reset", "Application has been reset.")

# Create tkinter home_window
home_window = tk.Tk()
home_window.title("Video Processing Application")

# Create widgets
label_name = tk.Label(home_window, text="Enter your name:")
entry_name = tk.Entry(home_window)
button_start = tk.Button(home_window, text="Start", command=operation_window)
button_reset = tk.Button(home_window, text="Reset", command=reset)

# Place widgets in the home_window
label_name.grid(row=0, column=0, padx=10, pady=5)
entry_name.grid(row=0, column=1, padx=10, pady=5)
button_start.grid(row=1, column=0, columnspan=2, padx=10, pady=5, sticky="WE")
button_reset.grid(row=2, column=0, columnspan=2, padx=10, pady=5, sticky="WE")

# Run the tkinter event loop
home_window.mainloop()


In [25]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox, filedialog

# Function to play video normally
def play_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Video', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to perform object detection on video
def detect_objects(video_path):
    net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
    classes = []
    with open('cfg/coco.names', 'r') as f:
        classes = f.read().splitlines()

    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (224, 224), swapRB=True, crop=False)
        net.setInput(blob)
        output_layers_names = net.getUnconnectedOutLayersNames()
        layer_outputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        if len(indexes) > 0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i], 2))
                color = colors[i]
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

        out.write(frame)
        cv2.imshow('Video with Object Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Function to reduce noise in the video
def reduce_noise(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        filtered_image = cv2.medianBlur(frame, 15)
        cv2.imshow('Video with Reduced Noise', filtered_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to sharpen the video
def sharpen_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened_image = cv2.filter2D(frame, -1, kernel)
        cv2.imshow('Sharpened Video', sharpened_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to adjust brightness and contrast of the video
def adjust_brightness_contrast(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        brightness = 5
        contrast = 1.5
        adjusted_image = cv2.addWeighted(frame, contrast, np.zeros(frame.shape, frame.dtype), 0, brightness)
        cv2.imshow('Video with Adjusted Brightness and Contrast', adjusted_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to perform operations based on user choice
def operation_window():
    name = entry_name.get()
    messagebox.showinfo("Welcome", f"Welcome {name}. This application allows you to perform various operations on a video.")
    home_window.destroy()
    
    operation_window = tk.Tk()   
    operation_window.title("Choose Operation")

    #label
    operation_label = tk.Label(operation_window, text="Select an operation you would like to perform:")
    operation_label.pack()

    #radio button
    operation_var = tk.IntVar()
    tk.Radiobutton(operation_window, text="Watch Video Normally", variable=operation_var, value=1).pack()
    tk.Radiobutton(operation_window, text="Perform Object Detection", variable=operation_var, value=2).pack()
    tk.Radiobutton(operation_window, text="Reduce Noise", variable=operation_var, value=3).pack()
    tk.Radiobutton(operation_window, text="Sharpen Video", variable=operation_var, value=4).pack()
    tk.Radiobutton(operation_window, text="Adjust Brightness and Contrast", variable=operation_var, value=5).pack()
    information_label = tk.Label(operation_window, text="You will be redirected to another window to select a video for your selected operation.")
    information_label.pack()

    #Button
    operation_button = tk.Button(operation_window, text="Proceed", command=lambda: perform_operation(operation_var.get(), operation_window))
    operation_button.pack()

def perform_operation(operation, operation_window):
    operation_window.destroy()
    if operation == 1:
        video_path = filedialog.askopenfilename(title="Select Video", filetypes=[("Video files", "*.mp4;*.avi")])
        play_video(video_path)
    elif operation == 2:
        video_path = filedialog.askopenfilename(title="Select Video", filetypes=[("Video files", "*.mp4;*.avi")])
        detect_objects(video_path)
    elif operation == 3:
        video_path = filedialog.askopenfilename(title="Select Video", filetypes=[("Video files", "*.mp4;*.avi")])
        reduce_noise(video_path)
    elif operation == 4:
        video_path = filedialog.askopenfilename(title="Select Video", filetypes=[("Video files", "*.mp4;*.avi")])
        sharpen_video(video_path)
    elif operation == 5:
        video_path = filedialog.askopenfilename(title="Select Video", filetypes=[("Video files", "*.mp4;*.avi")])
        adjust_brightness_contrast(video_path)

# Function to reset the application
def reset():
    entry_name.delete(0, tk.END)
    messagebox.showinfo("Reset", "Application has been reset.")

# Create tkinter home_window
home_window = tk.Tk()
home_window.title("Video Processing Application")

# Create widgets
label_name = tk.Label(home_window, text="Enter your name:")
entry_name = tk.Entry(home_window)
button_start = tk.Button(home_window, text="Start", command=operation_window)
button_reset = tk.Button(home_window, text="Reset", command=reset)

# Place widgets in the home_window
label_name.grid(row=0, column=0, padx=10, pady=5)
entry_name.grid(row=0, column=1, padx=10, pady=5)
button_start.grid(row=1, column=0, columnspan=2, padx=10, pady=5, sticky="WE")
button_reset.grid(row=2, column=0, columnspan=2, padx=10, pady=5, sticky="WE")

# Run the tkinter event loop
home_window.mainloop()


In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinter.ttk import Style, Button, Label, Entry, Radiobutton

# Function to play video normally
def play_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Video', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to perform object detection on video
def detect_objects(video_path):
    net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
    classes = []
    with open('cfg/coco.names', 'r') as f:
        classes = f.read().splitlines()

    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (224, 224), swapRB=True, crop=False)
        net.setInput(blob)
        output_layers_names = net.getUnconnectedOutLayersNames()
        layer_outputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        if len(indexes) > 0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i], 2))
                color = colors[i]
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

        out.write(frame)
        cv2.imshow('Video with Object Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Function to reduce noise in the video
def reduce_noise(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        filtered_image = cv2.medianBlur(frame, 15)
        cv2.imshow('Video with Reduced Noise', filtered_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to sharpen the video
def sharpen_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened_image = cv2.filter2D(frame, -1, kernel)
        cv2.imshow('Sharpened Video', sharpened_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to adjust brightness and contrast of the video
def adjust_brightness_contrast(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        brightness = 5
        contrast = 1.5
        adjusted_image = cv2.addWeighted(frame, contrast, np.zeros(frame.shape, frame.dtype), 0, brightness)
        cv2.imshow('Video with Adjusted Brightness and Contrast', adjusted_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Function to perform operations based on user choice
def operation_window():
    name = entry_name.get()
    
    # Create a new Toplevel window for welcome message
    welcome_window = tk.Toplevel()
    welcome_window.title("Welcome")
    
    # Create a Label with formatted text
    welcome_text = (
        f"Hello {name}.\n\n"
        f"Welcome to NWEJ's Video Editing and Detection Suite.\n\n"
        "This is an application that allows you to perform various operations on a video.\n\n"
        "Such operations include sharpening images, object detection, adjusting brightness and contrast etc."
    )
    label = tk.Label(welcome_window, text=welcome_text, font=("Arial", 12, "bold"), justify="center")
    label.pack(padx=20, pady=20)
    
    # Destroy the welcome window after a delay
    welcome_window.after(12000, welcome_window.destroy)
    
    # Close the home_window after the welcome message is displayed
    home_window.withdraw()
    
    # Create the operation window
    operation_window = tk.Toplevel()   
    operation_window.title("Choose Operation")
    operation_window.geometry("400x300")

    # Style
    style = Style()
    style.configure("Green.TButton", foreground="black", background="green")
    style.configure("Red.TButton", foreground="black", background="red")
    
    # Label
    operation_label = Label(operation_window, text=f"Select an operation you would like to perform:\n\n")
    operation_label.pack()

    # Radio button
    operation_var = tk.IntVar()
    operation_var.set(1)  # Default selection
    operations = [("Watch Video Normally", 1), ("Perform Object Detection", 2), ("Reduce Noise", 3), ("Sharpen Video", 4), ("Adjust Brightness and Contrast", 5)]
    for text, value in operations:
        Radiobutton(operation_window, text=text, variable=operation_var, value=value).pack(anchor='w')

    information_label = Label(operation_window, text=f"You will be redirected to another window to select a video for your selected operation.\n\n")
    information_label.pack()

    # Button
    operation_button = Button(operation_window, text="Proceed", style="Green.TButton", command=lambda: perform_operation(operation_var.get(), operation_window))
    operation_button.pack()





def perform_operation(operation, operation_window):
    operation_window.destroy()
    if operation:
        video_path = filedialog.askopenfilename(title="Select Video", filetypes=[("Video files", "*.mp4;*.avi")])
        if operation == 1:
            play_video(video_path)
        elif operation == 2:
            detect_objects(video_path)
        elif operation == 3:
            reduce_noise(video_path)
        elif operation == 4:
            sharpen_video(video_path)
        elif operation == 5:
            adjust_brightness_contrast(video_path)

# Function to reset the application
def reset():
    entry_name.delete(0, tk.END)
    messagebox.showinfo("Reset", "Application has been reset.")

# Create tkinter home_window
home_window = tk.Tk()
home_window.title("Video Processing Application")

# Create widgets
label_name = Label(home_window, text="Enter your name:")
entry_name = Entry(home_window)
button_start = Button(home_window, text="Start", style="Green.TButton", command=operation_window)
button_reset = Button(home_window, text="Reset", style="Red.TButton", command=reset)

# Place widgets in the home_window
label_name.grid(row=0, column=0, padx=10, pady=5)
entry_name.grid(row=0, column=1, padx=10, pady=5)
button_start.grid(row=1, column=0, columnspan=2, padx=10, pady=5, sticky="WE")
button_reset.grid(row=2, column=0, columnspan=2, padx=10, pady=5, sticky="WE")

# Run the tkinter event loop
home_window.mainloop()
